### __XDF Data Analysis of LSL timestamps (Unity + EEG)__
#### __TODO__
* [x] Read XDF file and header and select the right data (timestamps and values)
* [x] Compute the timestamps from 0
* [x] Visualize the data: unity audio vs microphone and unity color vs photodiode
* [x] Compare the timestamps (length, duration, sample count..): Original vs Calculated vs FileInfo
* [x] Descriptive statistics of timestamps distribution and plot
* [ ] Actual latency test: select the microphone and photodiode peaks (starting points) and compare with the unity ones
* [ ] Test all recordings
* [ ] Make and test long recordings (half an hour) and check with two computers (local network setup)
* [ ] Find out why sometimes Unity timestamps start before the EEG ones
* [ ] ...

#### __Dependencies__

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyxdf
from scipy.signal import find_peaks
import seaborn as sns

#### __Read XDF data__

In [12]:
streams, fileheader = pyxdf.load_xdf("data/ftest1.xdf")  # load the XDF file
fileheader  # only a dict describing the version and format of the XDF file

{'info': defaultdict(list, {'version': ['1.0']})}

#### __Read EEG and Unity timestamps and sensor data__

In [13]:
# Time values
a_ch = 0  # constant to select the unity audio stream
c_ch = 2  # constant to select the unity color stream
e_ch = 1  # constant to select the eeg stream
u_ts = streams[a_ch]["time_stamps"]  # unity timestamps
e_ts = streams[e_ch]["time_stamps"]  # eeg timestamps

# Diode values
eeg = np.transpose(streams[e_ch]["time_series"])
# select the photodiode and microphone sensor information
e_color = eeg[64]
e_audio = eeg[69]

# select unity audio and background color change markers
# format: [currentFrame, value, timestamp]
u_color = np.transpose(streams[c_ch]["time_series"])
u_audio = np.transpose(streams[a_ch]["time_series"])

e_color = -e_color  # invert diode data polarity

#### __Preprocess data: calculate meaningful timestamps__

In [14]:
# calculate time values for unity and eeg from 0
e_time = [0]
length = len(e_ts)
[e_time.append(e_ts[i + 1] - e_ts[0]) for i in range(length) if i < length - 1]

u_time = [0]
length = len(u_ts)
[u_time.append(u_ts[i + 1] - u_ts[0]) for i in range(length) if i < length - 1]

# calculate the diff and shift the values left (negative) or right (positive)
diff = u_ts[0] - e_ts[0]
u_time = [i + diff for i in u_time]

#### __Data preview__

In [15]:
%matplotlib widget
sns.set(rc={"figure.figsize": (14, 5)})  # set figure size
sns.set_style("darkgrid")  # set seaborn plotting style

# plot 5 seconds of diode and microphone raw data
five_sec = 1024 * 1  # N of eeg in 5 s
f_sec = 90 * 1  # N of unity in 5 s
plt.plot(e_time[:five_sec], e_color[:five_sec])
plt.plot(e_time[:five_sec], e_audio[:five_sec])
plt.plot(u_time[:f_sec], u_color[1][:f_sec] * 10000, marker="o")
plt.plot(u_time[:f_sec], u_audio[1][:f_sec] * 10000, marker="x")
plt.title(f"Sample: N = {five_sec}")
plt.ylabel("Sensor value")
plt.xlabel("Time (s)")
plt.xticks(np.arange(0, 1, step=0.05))
labels = ["photosensor", "microphone", "color", "audio"]
plt.legend(labels, loc="upper right")  # set the legend
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### __Timestamps comparison (original vs computed vs file info)__

In [16]:
# store unity and eeg timestamps as pandas series
# dataframe is not needed since it's 1D array

eeg_t = pd.Series(streams[e_ch]["time_stamps"])
unity_t = pd.Series(streams[a_ch]["time_stamps"])

print("Original timestamps")
print("===================")
u_start = u_ts[0]
u_end = u_ts[-1]
e_start = e_ts[0]
e_end = e_ts[-1]
u_length = u_end - u_start
e_length = e_end - e_start
print(f"EEG first timestamp: {e_start}")
print(f"EEG last timestamp: {e_end}")
print(f"EEG length: {e_length}")
print(f"EEG sample count: {len(e_ts)}")
print(f"Unity first timestamp: {u_start}")
print(f"Unity last timestamp: {u_end}")
print(f"Unity length: {u_length}")
print(f"Unity sample count: {len(u_ts)}")
print(f"Start difference: {abs(u_start - e_start)}")
print(f"Length difference: {abs(u_length - e_length)}")

print("")

print("Computed timestamps")
print("====================")
u_start = u_time[0]
# [-1:] returns the index and the type as well but [-1:].values[0] also works
u_end = u_time[-1]
e_start = e_time[0]
e_end = e_time[-1]
u_length = u_end - u_start
e_length = e_end - e_start
print(f"EEG first timestamp: {e_start}")
print(f"EEG last timestamp: {e_end}")
print(f"EEG length: {e_length}")
print(f"EEG sample count: {len(e_time)}")
print(f"Unity first timestamp: {u_start}")
print(f"Unity last timestamp: {u_end}")
print(f"Unity length: {u_length}")
print(f"Unity sample count: {len(u_time)}")
print(f"Start difference: {abs(u_start - e_start)}")
print(f"Length difference: {abs(u_length - e_length)}")

print("")

print("File info")
print("========")
e_info = streams[e_ch]["info"]
e_footer = streams[e_ch]["footer"]["info"]
u_info = streams[a_ch]["info"]
u_footer = streams[a_ch]["footer"]["info"]

print(f"EEG stream created at: {e_info['created_at'][0]}")
print(f"Unity stream created at: {u_info['created_at'][0]}")
print(f"EEG first timestamp: {e_footer['first_timestamp'][0]}")
print(f"EEG last timestamp: {e_footer['last_timestamp'][0]}")
print(f"EEG sample count: {e_footer['sample_count'][0]}")
print(f"Unity first timestamp: {u_footer['first_timestamp'][0]}")
print(f"Unity last timestamp: {u_footer['last_timestamp'][0]}")
print(f"Unity sample count: {u_footer['sample_count'][0]}")

Original timestamps
EEG first timestamp: 597683.7294010719
EEG last timestamp: 597805.7615867447
EEG length: 122.03218567278236
EEG sample count: 124960
Unity first timestamp: 597683.7352238147
Unity last timestamp: 597805.735361065
Unity length: 122.00013725028839
Unity sample count: 10982
Start difference: 0.005822742823511362
Length difference: 0.032048422493971884

Computed timestamps
EEG first timestamp: 0
EEG last timestamp: 122.03218567278236
EEG length: 122.03218567278236
EEG sample count: 124960
Unity first timestamp: 0.005822742823511362
Unity last timestamp: 122.0059599931119
Unity length: 122.00013725028839
Unity sample count: 10982
Start difference: 0.005822742823511362
Length difference: 0.032048422493971884

File info
EEG stream created at: 597605.9337288000
Unity stream created at: 597631.6023521000
EEG first timestamp: 597683.7440407
EEG last timestamp: 597805.7442204
EEG sample count: 124959
Unity first timestamp: 597683.7352271
Unity last timestamp: 597805.7353644
Un

#### __Descriptive statistics: EEG timestamps__

In [17]:
e_time_dist = [e_ts[i + 1] - e_ts[i] for i in range(len(e_ts) - 1)]
u_time_dist = [u_ts[i + 1] - u_ts[i] for i in range(len(u_ts) - 1)]

e_time_dist = pd.DataFrame(np.array(e_time_dist), columns=["eeg"])
u_time_dist = pd.DataFrame(np.array(u_time_dist), columns=["unity"])

e_time_dist.describe()

,eeg
count,1.249590e+05
mean,9.765778e-04
std,5.792683e-11
min,9.765778e-04
25%,9.765778e-04
50%,9.765778e-04
75%,9.765779e-04
max,9.765779e-04


The EEG samples look really constant over time

#### __Descriptive statistics: Unity timestamps__

In [18]:
u_time_dist.describe()

,unity
count,10981.000000
mean,0.011110
std,0.001141
min,0.000023
25%,0.011022
50%,0.011109
75%,0.011195
max,0.022469


It does not seem the case for the unity samples
#### __Time sampling plot comparison__

In [22]:
%matplotlib widget
sns.set(rc={"figure.figsize": (3, 9)})  # set figure size
sns.set_style("whitegrid")  # set seaborn plotting style
p = sns.boxplot(x=u_time_dist, orient="v")
p.set_title("Time distribution (s)")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### __Timestamps visualization in 1-second epochs__ (extra plots? not finished)

In [20]:
%matplotlib inline
# for t in range(1, int(eeg_t[eeg_t.last_valid_index()]) + 1):
#     # select and plot eeg
#     selected = eeg_t[eeg_t.between(t - 1, t)]
#     eeg_l = len(selected)  # sample count for eeg
#     sns.scatterplot(data=selected, marker="o", s=50)
#     # select and plot unity
#     selected = unity_t[unity_t.between(t - 1, t)]
#     unity_l = len(selected)  # sample count for eeg
#     p = sns.scatterplot(data=selected, marker="d", s=150)
#     p.set_title(f"Between {t-1}s and {t}s: eegN={eeg_l} unityN={unity_l}")
#     plt.legend(labels=["eeg", "unity"])  # set the legend
#     plt.show()  # display plot
    